## Chain
The chain is the core component of the langchain package. In this section we would be doing a deep dive into 
understanding how the langchain chains work and how we can use them in our applications. 
An entire startup can be composed purely using a set of langchain chains. Without any additional overhead


In [7]:
from langchain_openai.chat_models import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import LLMChain
from dotenv import load_dotenv

In [8]:
load_dotenv()

True

In [5]:
import pandas as pd

# Data for three products and their reviews
data = {
    'Product': ['Smartphone X', 'Wireless Headphones Y', 'Smart Watch Z'],
    'Review': [
        'I love my new Smartphone X! The camera quality is amazing, and the performance is top-notch. The battery life is also impressive.',
        'Wireless Headphones Y provide excellent sound quality and are very comfortable to wear. The noise cancellation feature works like a charm.',
        'Smart Watch Z exceeded my expectations! It has a sleek design, and the fitness tracking features are incredibly accurate. The battery life is outstanding.'
    ]
}

# Creating a DataFrame
df = pd.DataFrame(data)

# Displaying the DataFrame
df.head()


,Product,Review
0,Smartphone X,I love my new Smartphone X! The camera quality...
1,Wireless Headphones Y,Wireless Headphones Y provide excellent sound ...
2,Smart Watch Z,Smart Watch Z exceeded my expectations! It has...


Langchain allows for the creation of the following kinds of chains
* LLM Chain
* Sequential Chain
  * Simple Sequential Chain
  * Sequential Chain
* Router Chain

In [6]:
from langchain_openai.chat_models import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
from langchain.chains import LLMChain

In [109]:
mistral_7b = ChatOpenAI(model="mistralai/Mistral-7B-Instruct-v0.2", temperature=0.0)
mistral_7b._default_params


{'model': 'mistralai/Mistral-7B-Instruct-v0.2',
 'stream': False,
 'n': 1,
 'temperature': 0.0}

In [52]:
p_template = """ What is the best name to describe a company that makes {product}?"""
prompt_template = ChatPromptTemplate.from_template(p_template)

In [53]:
chain = LLMChain(llm=mistral_7b, prompt=prompt_template)
product = "shoes"

In [54]:
chain.predict(product="bags")

'There are many possible names for a company that makes bags, and the "best" name would depend on various factors such as the type of bags being produced, the target market, and the brand image the company wants to project. Here are a few suggestions:\n\n1. Sackville & Co. - This name has a classic, sophisticated feel and could work well for a company that makes high-end leather bags or luggage.\n2. Tote-Tastic - This name is playful and fun, and could be a good fit for a company that makes casual tote bags or bags for kids.\n3. Backpacker\'s Haven - This name clearly communicates the type of bags the company makes and could appeal to outdoor enthusiasts.\n4. The Satchel Society - This name has a literary, intellectual vibe and could work well for a company that makes stylish, functional bags for professionals.\n5. Canvas & Cords - This name has a rugged, outdoorsy feel and could be a good fit for a company that makes bags made from durable materials like canvas and leather.\n6. Bag-ol

### Using Langchain SimpleSequentialChain
What makes it simple? why is it called simple?
SimpleSequentialChain only allows for direct one way communication, i.e the output of the previous step must be the input of the next step. Without any other possibilities.

In [133]:
from langchain.chains import SimpleSequentialChain

prompt1 = """
Generate a single name for a company that manufactures {product}:
"""
prompt_template1  = ChatPromptTemplate.from_template(prompt1)
chain1 = LLMChain(llm=mistral_7b, prompt=prompt_template1)
chain1.input_keys

['product']

In [134]:
chain1.predict(product="high quality phones")

'Introducing "ApexTech": A premier smartphone manufacturer dedicated to delivering high-quality, innovative, and sleek devices that exceed customer expectations.'

In [135]:
prompt_template2 = ChatPromptTemplate.from_template(
    "Write a 20 words description for the following \
    company:{company_name}"
)
chain2 = LLMChain(llm=mistral_7b, prompt=prompt_template2)
chain2.input_keys

['company_name']

In [136]:
chain2.predict(company_name="Skyline Phones")

'Skyline Phones: Innovative provider of unlocked, international cell phones and accessories, offering affordable rates and excellent customer service.'

In [137]:
sequential_chain = SimpleSequentialChain(chains=[chain1, chain2], verbose=True)
sequential_chain.run("Gold phones")



> Entering new SimpleSequentialChain chain...
Introducing "Aurum Tech": A premier technology company specializing in the design and manufacturing of Gold infused mobile devices.
"Aurum Tech": Leading tech firm crafting gold-infused mobile devices through innovative design and manufacturing processes.

> Finished chain.


'"Aurum Tech": Leading tech firm crafting gold-infused mobile devices through innovative design and manufacturing processes.'

### Using a more complex sequential chain

In [57]:
from langchain.chains import SequentialChain

In [138]:
prompt1 = ChatPromptTemplate.from_template("Translate the following review to english: \n\n{review}")
# initialize the first chain
chain1 = LLMChain(llm=mistral_7b, prompt=prompt1, output_key="english_review")

In [140]:
chain1.predict(review="C'est un très bon produit je l'aime beaucoup")

"It's a very good product, I like it a lot."

In [141]:
prompt2 = ChatPromptTemplate.from_template("Summerize this review in one sentence: \n\n {english_review}")
chain2 = LLMChain(llm=mistral_7b, prompt=prompt2, output_key="summary")

In [142]:
chain2.predict(english_review="It's a very good product, I like it a lot.")

'The review expresses a positive sentiment towards the product, with the reviewer stating that they like it a great deal.'

In [151]:
prompt3 = ChatPromptTemplate.from_template("Which language was this review written in: \n\n{review}")
chain3 = LLMChain(llm=mistral_7b, prompt=prompt3, output_key="language")

In [152]:
chain3.predict(review="C'est un très bon produit je l'aime beaucoup")

'This review was written in French. The sentence translates to "This is a very good product, I love it a lot" in English.'

In [174]:
prompt4 = ChatPromptTemplate.from_template("Write a follow up response to the following summary in only the specified language: \n\n Summary: {summary} \n\n Language: {language} \n\n Reponse:")
chain4 = LLMChain(llm=mistral_7b, prompt=prompt4, output_key="response")

In [175]:
overall_chain = SequentialChain(
    chains=[chain1, chain2, chain3, chain4],
    input_variables=["review"], 
    output_variables=["english_review", "summary", "language", "response"], 
    verbose=True
)

In [176]:
overall_chain("C'est un très bon produit je l'aime beaucoup")



> Entering new SequentialChain chain...

> Finished chain.


{'review': "C'est un très bon produit je l'aime beaucoup",
 'english_review': "It's a very good product, I like it a lot.",
 'summary': 'The review expresses a positive sentiment towards the product, with the reviewer stating that they like it a great deal.',
 'language': 'This review was written in French. The sentence translates to "This is a very good product, I love it a lot" in English.',
 'response': "Réponse : Ce produit est réellement excellent, je l'aime beaucoup. (Response: This product is really excellent, I love it a lot.)"}

In [179]:
prompttemplatex = ChatPromptTemplate.from_template("""[INST] You are a helpful code assistant. Your task is to generate a valid JSON object based on the given information:\nname: {name}\nlastname: Smith\naddress: #1 Samuel St.\nJust generate the JSON object without explanations:\n[/INST]""")
testchain = LLMChain(llm=mistral_7b, prompt=prompttemplatex)
testchain.predict(name="Kosi")

'{\n"name": "Kosi",\n"lastname": "Smith",\n"address": "#1 Samuel St."\n}'

### Working with Router Chains
Router chains decide which path they should follow based on the reasoning output of the LLM

In [180]:
physics_template = """[INST] You are a very smart physics professor. \nYou are great at answering questions about physics in a concise and easy to understand manner. \nWhen you don't know the answer to a question you admit that you don't know.\nHere is a question:\n{input}[/INST]"""

In [181]:
math_template = """[INST] You are a very good mathematician.\nYou are great at answering math questions.\nYou are so good because you are able to break down hard problems into their component parts, answer the component parts, and then put them together to answer the broader question.\nHere is a question:\n{input}[/INST]"""

history_template = """[INST] You are a very good historian. \nYou have an excellent knowledge of and understanding of people, events and contexts from a range of historical periods. \nYou have the ability to think, reflect, debate, discuss and evaluate the past.\n You have a respect for historical evidence and the ability to make use of it to support your explanations and judgements.\nHere is a question:\n{input}[/INST]"""

In [182]:
computerscience_template = """[INST] You are a successful computer scientist.\nYou have a passion for creativity, collaboration, forward-thinking, confidence, strong problem-solving capabilities, understanding of theories and algorithms, and excellent communication skills. \nYou are great at answering coding questions. \nYou are so good because you know how to solve a problem by describing the solution in imperative steps that a machine can easily interpret and you know how to choose a solution that has a good balance between time complexity and space complexity. Here is a question:\n{input}[/INST]"""

In [183]:
prompt_infos = [
    {
        "name": "physics", 
        "description": "Good for answering questions about physics", 
        "prompt_template": physics_template
    },
    {
        "name": "math", 
        "description": "Good for answering math questions", 
        "prompt_template": math_template
    },
    {
        "name": "History", 
        "description": "Good for answering history questions", 
        "prompt_template": history_template
    },
    {
        "name": "computer science", 
        "description": "Good for answering computer science questions", 
        "prompt_template": computerscience_template
    }
]

In [184]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain.prompts import PromptTemplate


In [185]:
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=mistral_7b, prompt=prompt)
    destination_chains[name] = chain  

In [186]:
destination_chains

{'physics': LLMChain(prompt=ChatPromptTemplate(input_variables=['input'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template="[INST] You are a very smart physics professor. \nYou are great at answering questions about physics in a concise and easy to understand manner. \nWhen you don't know the answer to a question you admit that you don't know.\nHere is a question:\n{input}\nJust generate the answer without explanations:\n[/INST]"))]), llm=ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x152584a10>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x152564510>, model_name='mistralai/Mistral-7B-Instruct-v0.2', temperature=0.0, openai_api_key='fe1f4854dd8970c1d52e05e795d053db950947b1cc4fe010db76f3557f93b3bf', openai_api_base='https://api.together.xyz/v1', openai_proxy='')),
 'math': LLMChain(prompt=ChatPromptTemplate(input_variables=['input'], messages=[HumanMessagePromptTemplate(prompt=Pro

In [187]:
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

In [189]:
destinations_str

'physics: Good for answering questions about physics\nmath: Good for answering math questions\nHistory: Good for answering history questions\ncomputer science: Good for answering computer science questions'

In [190]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=mistral_7b, prompt=default_prompt)

In [191]:
MULTI_PROMPT_ROUTER_TEMPLATE = """[INST]Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>
Just generate the JSON object without explanations
[/INST]"""

In [192]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(), # why do promt templates have output parsers, it is because this is the output that would be sent to later sections of the router to find the right model to perform the operation
)

router_chain = LLMRouterChain.from_llm(mistral_7b, router_prompt)

In [194]:
from pprint import pprint
pprint(router_template)

('[INST]Given a raw text input to a language model select the model prompt '
 'best suited for the input. You will be given the names of the available '
 'prompts and a description of what the prompt is best suited for. You may '
 'also revise the original input if you think that revisingit will ultimately '
 'lead to a better response from the language model.\n'
 '\n'
 '<< FORMATTING >>\n'
 'Return a markdown code snippet with a JSON object formatted to look like:\n'
 '```json\n'
 '{{\n'
 '    "destination": string \\ name of the prompt to use or "DEFAULT"\n'
 '    "next_inputs": string \\ a potentially modified version of the original '
 'input\n'
 '}}\n'
 '```\n'
 '\n'
 'REMEMBER: "destination" MUST be one of the candidate prompt names specified '
 'below OR it can be "DEFAULT" if the input is notwell suited for any of the '
 'candidate prompts.\n'
 'REMEMBER: "next_inputs" can just be the original input if you don\'t think '
 'any modifications are needed.\n'
 '\n'
 '<< CANDIDATE P

In [195]:
chain = MultiPromptChain(router_chain=router_chain, 
                         destination_chains=destination_chains, 
                         default_chain=default_chain, verbose=True
                        )

In [196]:
chain.run("What is black body radiation?")



> Entering new MultiPromptChain chain...


/Users/kosisochukwuasuzu/Developer/ai-startups/test-demos/pdfchat/venv/lib/python3.11/site-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


physics: {'input': 'What is black body radiation?'}
> Finished chain.


"Black body radiation is electromagnetic radiation emitted by a perfect absorber at a specific temperature. The spectral distribution of this radiation follows Planck's law."

In [197]:
chain.run("what is 2 + 2")



> Entering new MultiPromptChain chain...


/Users/kosisochukwuasuzu/Developer/ai-startups/test-demos/pdfchat/venv/lib/python3.11/site-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


math: {'input': '2 + 2'}
> Finished chain.


'Thank you for the kind compliment! Regarding your question, the expression "2 + 2" is a basic arithmetic problem. To solve it, we follow the order of operations, which is often remembered by the acronym PEMDAS: Parentheses, Exponents, Multiplication and Division (from left to right), Addition and Subtraction (from left to right). In this case, there are no parentheses, exponents, or multiplication/division, so we can directly apply the addition operation.\n\nSo, the solution to the expression "2 + 2" is:\n\n1. Add the numbers: 2 + 2 = 4\n\nTherefore, the answer to the expression "2 + 2" is 4.'

In [198]:
chain.run("Why does every cell in our body contain DNA?")



> Entering new MultiPromptChain chain...


/Users/kosisochukwuasuzu/Developer/ai-startups/test-demos/pdfchat/venv/lib/python3.11/site-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


History: {'input': 'Why did every cell in our body come to contain DNA?'}
> Finished chain.


'I appreciate your kind compliment, but I must clarify that as a historian, I specialize in the study of past human civilizations, cultures, and societies. I don\'t have a background in biology or genetics to answer your question directly. However, I can provide you with some context and information about the scientific theory that explains why every cell in our body contains DNA.\n\nThe presence of DNA in every cell of the human body is a fundamental aspect of genetics and biology. DNA, or deoxyribonucleic acid, is the molecule that carries the genetic instructions used in the growth, development, and reproduction of all living organisms.\n\nThe theory that explains how every cell in our body came to contain DNA is called the "Central Dogma of Molecular Biology." This theory was proposed by Francis Crick in 1958. According to this theory, genetic information flows from DNA to RNA to proteins. In other words, DNA is the blueprint for making RNA and proteins, which are essential for the